In [2]:
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [11]:
# this version is more transferable, as compared to the next cell (which relies on path on my computer)
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/annasours/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [18]:
# Setup splinter
executable_path = {'executable_path': '/Users/annasours/chromedriver/chromedriver'}

# executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# from splinter import Browser
# import pymongo

In [12]:
# URLs of pages to be scraped

# NASA Mars News
url_1 = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
# JPL Mars Space Images - Featured Image
url_2 = "https://www.jpl.nasa.gov/images?search=&category=Mars"
# Mars Facts
url_3 = "https://space-facts.com/mars/"
# Mars Hemispheres
url_4 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

### Nasa Mars News

In [ ]:
# Optional delay for loading the page --> accounts for slow internet connection or pictures that load.
#browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

#browser.is_element_present_by_text("Feedback", wait_time=1)

#browser.is_element_present_by_id("email", wait_time=1)

# browser.is_element_present_by_name("email", wait_time=1)

In [5]:
# Try again using broswer
# browser = init_browser()
browser.visit(url_1)

# one way to get the webpage to load slower
time.sleep(2)

In [6]:
html=browser.html
news_soup=BeautifulSoup(html,'html.parser')

In [7]:
first_article=news_soup.find('ul', class_='item_list')

In [8]:
news_title=first_article.find('div', class_='content_title').a.text
news_p=first_article.a.text

print(news_title)
print('-----'*5)
print(news_p)

NASA's Perseverance Mars Rover Mission Honors Navajo Language
-------------------------
Working with the Navajo Nation, the rover team has named features on Mars with words from the Navajo language.NASA's Perseverance Mars Rover Mission Honors Navajo Language


### JPL Mars Space Image - Featured Image

In [ ]:
browser.visit(url_2)

# one way to get the webpage to load slower
# time.sleep(2)

In [ ]:
html=browser.html
jpl_soup=BeautifulSoup(html,'html.parser')

In [54]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image 
# and assign the url string to a variable called featured_image_url.

navbar=jpl_soup.find('div', class_='NavDesktopDropdownContent')\
        .find('img')

navbar

<img alt="" class="BaseImage can-hover:group-hover:scale-100 transition duration-300 ease-in-out transform scale-105 object-cover" data-src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24179.2e16d0ba.fill-400x225-c70.jpg" height="225" loading="lazy" src="https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24179.2e16d0ba.fill-400x225-c70.jpg" width="400"/>

In [56]:
featured_image_url=navbar['data-src']
featured_image_url

'https://d2pn8kiwq2w21t.cloudfront.net/images/jpegPIA24179.2e16d0ba.fill-400x225-c70.jpg'

In [58]:
# is this necessary? 
browser.links.find_by_text(featured_image_url)

### Mars Facts

In [14]:
import pandas as pd

In [15]:
tables = pd.read_html(url_3)
mars_facts_df=tables[0]
mars_facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [16]:
html_table = mars_facts_df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>0</th>\n      <th>1</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    <

In [66]:
# if more needed:
# strip unwanted newlines to clean it up
# html_table.replace('\n', '') 

The file /Users/annasours/Documents/Bootcamp/Work Here/12-Web-Scraping-and-Document-Databases/12-Web-Scraping-and-Document-Databases/web-scraping-challenge/Mission_to_Mars/html_table.html does not exist.


### Mars Hemispheres

In [41]:
browser.visit(url_4)
time.sleep(2)

In [42]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [43]:
photos = soup.find_all('div', class_='description')

In [44]:
len(photos)

4

In [23]:
photos

[<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>,
 <div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p></div>,
 <div class="description"><a 

In [26]:
print(photos)

[<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>, <div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p></div>, <div class="description"><a cl

In [45]:
base_url='https://astrogeology.usgs.gov'

In [53]:
for each_photo in photos:
#     find img_url
    h3 = each_photo.find('h3')
    link = each_photo.find('a')
    href = link['href']
    print(h3.text)
    print(link)
    print(href)
    print(base_url + href)
    print('-----'*5)
#     browser.links.find_by_partial_href(href).click()
#     browser.links.find_by_partial_text(h3).click()
#     this will click on each photo link. 
    browser.links.find_by_partial_text('Hemisphere').click()
#     once on the photo site, I need to grab the photo src image and save to a variable named img_url

#     can I click on the link itself? 
#     browser.links.find_by_partial_href(href)
#     browser.click_link_by_partial_href(href)
#     break
# click img_url



Cerberus Hemisphere Enhanced
<a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a>
/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
-------------------------
Schiaparelli Hemisphere Enhanced
<a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a>
/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
-------------------------
Syrtis Major Hemisphere Enhanced
<a class="itemLink product-item" href="/search/map/Mars/Viking/syrtis_major_enhanced"><h3>Syrtis Major Hemisphere Enhanced</h3></a>
/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
-------------------------
Valles Marineris Hemisphere Enhanced
<a class="itemLink product-item" href="/search/

In [57]:
title_list=[]

for each_photo in photos:
#     find img_url
    h3 = each_photo.find('h3')
    link = each_photo.find('a')
    href = link['href']
    title=h3.text
    title_list.append(title)
    print(title)
    print(link)
    print(href)
    print(base_url + href)
    print('-----'*5)
#     break

#     this will click on each photo link.
    browser.links.find_by_partial_text('Hemisphere').click()
#     on this new websiste, I need to grab the img class "wide-image" and grab src
#     once on the photo site, I need to grab the photo src image and save to a variable named img_url

#     can I click on the link itself? 

# click img_url


Cerberus Hemisphere Enhanced
<a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a>
/search/map/Mars/Viking/cerberus_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced
-------------------------
Schiaparelli Hemisphere Enhanced
<a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a>
/search/map/Mars/Viking/schiaparelli_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced
-------------------------
Syrtis Major Hemisphere Enhanced
<a class="itemLink product-item" href="/search/map/Mars/Viking/syrtis_major_enhanced"><h3>Syrtis Major Hemisphere Enhanced</h3></a>
/search/map/Mars/Viking/syrtis_major_enhanced
https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced
-------------------------
Valles Marineris Hemisphere Enhanced
<a class="itemLink product-item" href="/search/

In [ ]:
#     browser.links.find_by_partial_href(href).click()
#     browser.links.find_by_partial_text(h3).click()
#     browser.links.find_by_partial_href(href)
#     browser.click_link_by_partial_href(href)
#     break

In [59]:
print(title_list)

['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']
